# MACHINE LEARNING TERM PROJECT
## FAKE NEWS PREDICTION
### AHMAD MAKKI - MSDSF23M022

Fake News Prediction Project Details Here

Data Set Details:
1. id - Unique ID of news article
2. title - Title of news article
3. author: Author of the news article
4. text: Description of the news article
5. label: The News is fake or real
    1. Fake News: 1 
    2. Real News: 0

Data Set Kaggle Link:
https://www.kaggle.com/c/fake-news/data?select=train.csv

### Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Load the data set

In [2]:
news_dataset=pd.read_csv("train.csv")

In [3]:
news_dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# To check how many spaces is empty
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Replace blanck spaces with empty string ''
news_dataset=news_dataset.fillna('')

In [7]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
# Merging Title and Author Couloumn into Contents 
news_dataset['contents']=news_dataset['title'] + ' ' + news_dataset['author']
news_dataset['contents']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: contents, Length: 20800, dtype: object

In [9]:
# Seperate the Label Coloumn as well and seperate it
X=news_dataset.drop("label",axis=1) #coloumn drop
Y=news_dataset["label"]
X

,id,title,author,text,contents
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal..."


In [10]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ahmad
[nltk_data]     Makki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
print(stopwords.words('english')) #words list ehich don't have value to our textual data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Stemming Function

In [21]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [22]:
news_dataset['contents']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: contents, Length: 20800, dtype: object

In [24]:
news_dataset['contents']=news_dataset['contents'].apply(stemming)

In [25]:
news_dataset['contents']

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: contents, Length: 20800, dtype: object

In [26]:
X=news_dataset['contents'].values
Y=news_dataset['label'].values

In [27]:
print(X)
print(X.shape)

['hous dem aid even see comey letter jason chaffetz tweet darrel lucu'
 'flynn hillari clinton big woman campu breitbart daniel j flynn'
 'truth might get fire consortiumnew com' ...
 'maci said receiv takeov approach hudson bay new york time michael j de la merc rachel abram'
 'nato russia hold parallel exercis balkan alex ansari'
 'keep f aliv david swanson']
(20800,)


In [28]:
print(Y)
print(Y.shape)

[1 0 1 ... 0 1 1]
(20800,)


Transforming Textual Data into Numeric Data

In [29]:
vectorizer=TfidfVectorizer()
X=vectorizer.fit_transform(X)

In [30]:
print(X)

  (0, 8909)	0.3635963806326075
  (0, 3600)	0.3598939188262559
  (0, 15686)	0.28485063562728646
  (0, 2483)	0.3676519686797209
  (0, 7692)	0.24785219520671603
  (0, 8630)	0.29212514087043684
  (0, 2959)	0.2468450128533713
  (0, 13473)	0.2565896679337957
  (0, 4973)	0.233316966909351
  (0, 267)	0.27010124977708766
  (0, 3792)	0.2705332480845492
  (0, 7005)	0.21874169089359144
  (1, 3568)	0.26373768806048464
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 16799)	0.30071745655510157
  (1, 1497)	0.2939891562094648
  (1, 2813)	0.19094574062359204
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 1287)	0.3353805680413986
  (20797, 13122)	0.24825263521976057
  (20797, 12344)	0.27263457663336677
  (20797, 14967)	0.3115945315488075
  (20797, 12138)	0.24778257724396505
  (20797, 9518)	0.2954204

In [31]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

## Train Model using LogisticRegression

In [32]:
log_reg=LogisticRegression()
log_reg.fit(X_train,Y_train)

LogisticRegression()

Prediction on Training Data

In [33]:
train_pred=log_reg.predict(X_train)
train_accuracy=accuracy_score(train_pred,Y_train)

In [34]:
print("Accuracy Score on Training Data:",train_accuracy)

Accuracy Score on Training Data: 0.9865985576923076


Prediction on Testing Data

In [35]:
test_pred=log_reg.predict(X_test)
test_accuracy=accuracy_score(test_pred,Y_test)

In [36]:
print("Accuracy Score on Testing Data:",test_accuracy)

Accuracy Score on Testing Data: 0.9790865384615385


## Predictive System for News

In [37]:
X_new=X_test[1000]
prediction=log_reg.predict(X_new)
print(prediction)

if (prediction==0):
    print("The news is Real.")
else:
    print("The news is Fake.")

[0]
The news is Real.


In [38]:
# to check wheather it's predicting right or not.
print(Y_test[1000])

0


## Train Model using Support Vector Machines (SVM) 

In [43]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_model = SVC()
svm_model.fit(X_train, Y_train)

# Predictions on training data
train_pred = svm_model.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred = svm_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)


Accuracy Score on Training Data: 0.9990985576923077
Accuracy Score on Testing Data: 0.9889423076923077


## Train Model using RandomForestClassifier

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

# Predictions on training data
train_pred = rf_model.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)

Accuracy Score on Training Data: 1.0
Accuracy Score on Testing Data: 0.9935096153846154


## Train Model using NaiveBaye's

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

nb_model = MultinomialNB()
nb_model.fit(X_train, Y_train)

# Predictions on training data
train_pred = nb_model.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred = nb_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)

Accuracy Score on Training Data: 0.9782451923076924
Accuracy Score on Testing Data: 0.9550480769230769


## Train Model using DecesionTreeClassifier

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, Y_train)

# Predictions on training data
train_pred = dt_model.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred = dt_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)

Accuracy Score on Training Data: 1.0
Accuracy Score on Testing Data: 0.9918269230769231


## Train Model using GradientBoostingClassifier

In [45]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, Y_train)

# Predictions on training data
train_pred = gb_model.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred = gb_model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)

Accuracy Score on Training Data: 0.9721153846153846
Accuracy Score on Testing Data: 0.9673076923076923


## Binary Classification Neural Network using TensorFlow/Keras

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

# Predictions on training data
train_pred_proba = model.predict(X_train)
train_pred = (train_pred_proba > 0.5).astype(int)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Accuracy Score on Training Data:", train_accuracy)

# Predictions on testing data
test_pred_proba = model.predict(X_test)
test_pred = (test_pred_proba > 0.5).astype(int)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Accuracy Score on Testing Data:", test_accuracy)




Epoch 1/10


520/520 [==============================] - 40s 56ms/step - loss: 0.2278 - accuracy: 0.9494 - val_loss: 0.0558 - val_accuracy: 0.9880
Epoch 2/10
520/520 [==============================] - 28s 54ms/step - loss: 0.0215 - accuracy: 0.9963 - val_loss: 0.0371 - val_accuracy: 0.9899
Epoch 3/10
520/520 [==============================] - 29s 56ms/step - loss: 0.0064 - accuracy: 0.9995 - val_loss: 0.0330 - val_accuracy: 0.9901
Epoch 4/10
520/520 [==============================] - 25s 48ms/step - loss: 0.0028 - accuracy: 0.9999 - val_loss: 0.0319 - val_accuracy: 0.9899
Epoch 5/10
520/520 [==============================] - 25s 49ms/step - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.0311 - val_accuracy: 0.9904
Epoch 6/10
520/520 [==============================] - 28s 53ms/step - loss: 8.7232e-04 - accuracy: 1.0000 - val_loss: 0.0310 - val_accuracy: 0.9901
Epoch 7/10
520/520 [==============================] - 28s 54ms/step - loss: 5.6496e-04 - accuracy: 1.0000 - val_loss: 0.0310 - 

In [47]:
import getpass

username = getpass.getuser()
print(f"Current session username: {username}")


Current session username: Ahmad Makki


In [49]:
# Simple Binary Classification with Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=20, batch_size=64, validation_data=(X_test, Y_test))

# Evaluate on training data
train_pred = (model.predict(X_train) > 0.5).astype(int)
train_accuracy = accuracy_score(Y_train, train_pred)
print("Training Accuracy:", train_accuracy)

# Evaluate on testing data
test_pred = (model.predict(X_test) > 0.5).astype(int)
test_accuracy = accuracy_score(Y_test, test_pred)
print("Testing Accuracy:", test_accuracy)

Epoch 1/20
260/260 [==============================] - 15s 54ms/step - loss: 0.3292 - accuracy: 0.9343 - val_loss: 0.1037 - val_accuracy: 0.9832
Epoch 2/20
260/260 [==============================] - 14s 55ms/step - loss: 0.0475 - accuracy: 0.9944 - val_loss: 0.0524 - val_accuracy: 0.9885
Epoch 3/20
260/260 [==============================] - 15s 57ms/step - loss: 0.0163 - accuracy: 0.9986 - val_loss: 0.0411 - val_accuracy: 0.9904
Epoch 4/20
260/260 [==============================] - 14s 54ms/step - loss: 0.0079 - accuracy: 0.9996 - val_loss: 0.0367 - val_accuracy: 0.9894
Epoch 5/20
260/260 [==============================] - 14s 55ms/step - loss: 0.0045 - accuracy: 0.9999 - val_loss: 0.0340 - val_accuracy: 0.9899
Epoch 6/20
260/260 [==============================] - 15s 57ms/step - loss: 0.0029 - accuracy: 0.9999 - val_loss: 0.0329 - val_accuracy: 0.9904
Epoch 7/20
260/260 [==============================] - 15s 56ms/step - loss: 0.0020 - accuracy: 0.9999 - val_loss: 0.0321 - val_accuracy: